In [1]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
!apt-get install poppler-utils libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-script-latn is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
python3-pil is already the newest version (9.0.1-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.


In [3]:
!pip install unstructured[all-docs] cmake python-dotenv pdf2image python-dateutil faiss-cpu sentence-transformers langchain==0.2.5 langchain-community==0.2.5 langchain-core==0.2.9 langchain-openai==0.1.9 bitsandbytes accelerate xformers triton transformers

In [4]:
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

In [8]:
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [5]:
import os

filename = "2401.00222v1.pdf"

elements = partition_pdf(filename,
              chunking_strategy='by_title',
              infer_table_structure=True,
              extract_image_block_types=['Image'],
              max_characters=4000,
              new_after_n_chars=3800,
              combine_text_under_n_chars=2000,
              strategy='hi_res')

- extract_images_in_pdf:
Only applicable if `strategy=hi_res`.
If True, any detected images will be saved in the path specified by
'extract_image_block_output_dir' or stored as base64 encoded data within metadata fields.
Deprecation Note: This parameter is marked for deprecation. Future versions will use
'extract_image_block_types' for broader extraction capabilities.

- extract_image_block_types:
Only applicable if `strategy=hi_res`.
Images of the element type(s) specified in this list (e.g., ["Image", "Table"]) will be
saved in the path specified by 'extract_image_block_output_dir' or stored as base64
encoded data within metadata fields.

In [6]:
partition_pdf?

In [7]:
# 影像的位置

os.listdir('figures')

['figure-5-4.jpg',
 'figure-8-5.jpg',
 'figure-10-5.jpg',
 'figure-8-3.jpg',
 'figure-9-6.jpg',
 'figure-10-7.jpg',
 'figure-5-2.jpg',
 'figure-3-2.jpg',
 'figure-3-3.jpg',
 'figure-9-4.jpg',
 'figure-11-6.jpg',
 'figure-3-1.jpg',
 'figure-11-8.jpg']

In [8]:
"""
- CompositeElement: 文字
- Table: 表格
"""

elements

In [9]:
import configparser

from langchain.chat_models import ChatOpenAI


def credential_init():

  credential_file = "credentials.ini"

  if os.path.exists(credential_file):
      credentials = configparser.ConfigParser()
      credentials.read(credential_file)
      os.environ['OPENAI_API_KEY'] = credentials['openai'].get('api_key')
  else:
      os.environ['OPENAI_API_KEY'] = os.environ['OPENAI']

credential_init()


model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
           model_name="gpt-4o-2024-05-13", temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
prompt = f"Summarize the following text:\n\n{elements[0]}\n\nSummary:"

In [11]:
model.invoke(prompt)

AIMessage(content='The study by Shihan Zhao and Jian Tang from Sun Yat-sen University focuses on optimizing the yield of muonium in perforated silica aerogel for muonium-to-antimuonium conversion experiments, which are crucial for exploring new physics beyond the Standard Model. Through simulations validated by experimental data, the researchers optimized the target geometry, achieving a maximum muonium emission efficiency of 7.92% and a maximum vacuum yield of 1.134% with a typical surface muon beam. These results represent significant enhancements of 2.6 times and 2.1 times, respectively, over previous yields, potentially advancing future muonium experiments.', response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 313, 'total_tokens': 442, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt

In [12]:
type(elements[0])

unstructured.documents.elements.CompositeElement

In [13]:
elements[0].text

'3\n\n2 0 2 c e D 0 3 ] x e - p e h [ 1 v 2 2 2 0 0 . 1 0 4 2 : v i\n\nX\n\nr\n\na\n\nOptimization of muonium yield in perforated silica aerogel\n\nShihan Zhao1 and Jian Tang1, ∗\n\n1School of Physics, Sun Yat-sen University, Guangzhou 510275, China\n\n(Dated: January 2, 2024)\n\nA muonium consists of a positive muon associated with an orbital electron, and the spontaneous conversion to antimuonium serves as a clear indication of new physics beyond the Standard Model in particle physics. One of the most important aspects in muonium-to-antimuonium conversion experiment (MACE) is to increase the muo- nium yield in vacuum to challenge the latest limit obtained in 1999. This study focuses on a simulation of the muonium formation and diffusion in the perforated silica aerogel. The independent simulation results can be well validated by exper- imental data. By optimizing the target geometry, we find a maximum muonium emission efficiency of 7.92(2)% and a maximum vacuum yield of 1.134(2)% wit

In [14]:
prompt = f"Summarize the following text:\n\n{elements[7]}\n\nSummary:"
model.invoke(prompt)

AIMessage(content='The text discusses the simulation of vacuum muonium emission efficiency from perforated targets with various ablation schemes, comparing the results with experimental data from TRIUMF. The simulation setup mirrors the experiment, using a collimated surface muon beam to hit the target, and measures muonium decays to calculate emission efficiency. The results show a general agreement with experimental data, with most errors within 20%, though some deviations are noted and attributed to potential systematic errors. The validated simulation method is then used to optimize muonium yield by studying different target geometries and beam conditions. Various parameters such as hole spacing, diameter, and depth are explored to find optimal configurations for maximum muonium emission efficiency.', response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 948, 'total_tokens': 1087, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens':

## 要如何判別文字或是表格?

In [15]:
str(type(elements[0]))

"<class 'unstructured.documents.elements.CompositeElement'>"

In [17]:
str(type(elements[8]))

"<class 'unstructured.documents.elements.Table'>"

## 影像，表格，文字 三位一體 檢索系統

一個簡單的範例，你當然可以做得很複雜。像是整合text的部分然後用Semantic Splitting拆分。

In [18]:
import io
import base64

from PIL import Image
from langchain.docstore.document import Document
from langchain_core.messages.human import HumanMessage


def image_to_base64(image_path):

  with Image.open(image_path) as image:
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    image_str = base64.b64encode(buffered.getvalue())
  return image_str.decode('utf-8')


def get_summary(element, model):

  str_type = str(type(element))

  if 'CompositeElement' in str_type:
    prompt = f"Summarize the following text:\n\n{element}\n\nSummary:"
  if 'Table' in str_type:
    prompt = f"Summarize the following table:\n\n{element}\n\nSummary:"

  response = model.invoke(prompt)

  return response.content


def get_image_summary(filename, model):

  image_str = image_to_base64(filename)

  human_message = HumanMessage(content=[{'type': 'text',
                        'text': 'What is in this image?'},
                        {'type': 'image_url',
                         'image_url': {
                          'url': f"data:image/png;base64,{image_str}"}
                        }])

  response = model.invoke([human_message])

  return response.content


In [19]:
documents = []


for element in elements:

  str_type = str(type(element))

  summary = get_summary(element, model)

  if 'CompositeElement' in str_type:
    type_ = 'text'
  if 'Table' in str_type:
    type_ = 'table'

  documents.append(Document(page_content=summary, metadata={'type': type_}))

In [21]:
for image_file in os.listdir('figures'):

  image_path = f'figures/{image_file}'

  summary = get_image_summary(image_path, model)

  documents.append(Document(page_content=summary, metadata={'type': 'image', 'filename': image_path}))

In [22]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
image_retriever = vectorstore.as_retriever(search_kwargs={"filter": {'type': 'image'}})

In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


retriever = vectorstore.as_retriever()

template = """
Answer the question based only on the following context, which can include text, images, and tables:
{context}
Question: {question}

"""

prompt = ChatPromptTemplate.from_template(template)

chain = {"context": image_retriever, 'question': RunnablePassthrough()}| prompt| model| StrOutputParser()

In [31]:
chain.invoke("What is suggested by the images?")

'The images suggest a close-up view of a perforated surface with evenly spaced holes. The terms "spacing" and "diameter" are labeled, indicating the distance between the centers of adjacent holes and the width of the holes, respectively. The surface is illuminated, creating a pattern of light and shadow around the holes.'

In [33]:
image_retriever.invoke("What is suggested by the images?")

[Document(page_content='The image shows a close-up view of a perforated surface with evenly spaced holes. The terms "spacing" and "diameter" are labeled on the image, indicating the distance between the centers of adjacent holes (spacing) and the width of the holes (diameter). The surface appears to be illuminated, creating a pattern of light and shadow around the holes.', metadata={'type': 'image', 'filename': 'figures/figure-3-2.jpg'})]

In [28]:
image_retriever = vectorstore.as_retriever(search_kwargs={"filter": {'type': 'image'}, 'k': 1})

In [29]:
len(image_retriever.invoke("What is suggested by the images?"))

4